In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir('/kaggle/input/vegetable-image-dataset/Vegetable Images/'))

# for dirname, _, filenames in os.walk('/kaggle/input/kaggle/input/vegetable-image-dataset/Vegetable Images/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import tensorflow as tf

In [4]:
batch_size = 32
img_height = 180
img_width = 180
data_dir = '/kaggle/input/vegetable-image-dataset/Vegetable Images/'

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir+'/train',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir+'/validation',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [7]:
class_names = train_ds.class_names
print(class_names)

In [8]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [9]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [10]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [11]:
num_classes = 15

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [12]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [13]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

In [14]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(3)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [66]:
image_url = "https://www.healthifyme.com/blog/wp-content/uploads/2020/01/capsicum-cover-1.jpg"
image_path = tf.keras.utils.get_file('/kaggle/working/Vegetable1.jpg', origin=image_url,cache_dir=None,cache_subdir='')

In [67]:
img = tf.keras.utils.load_img(
    image_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [57]:
model.save('my_model.h5')
# Install helper packages:
!pip install tf2onnx onnx onnxruntime

# Load model from .h5 and save as Saved Model:
import tensorflow as tf
model = tf.keras.models.load_model("my_model.h5")
tf.saved_model.save(model, "tmp_model")

# Convert in bash:
!python -m tf2onnx.convert --saved-model tmp_model --output "my_model.onnx"

In [19]:
tfjs.converters.save_keras_model(model, ".")